In [122]:
import torch
import torchvision
import torchvision.transforms as transforms
import pandas as pd
import numpy as np

In [4]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
train = pd.read_csv('kaggle_data/train.csv')

In [4]:
train_small = train.iloc[:10000]

In [68]:
train_small = train.loc[np.random.choice(train.index, size=800000, replace=False)]

In [69]:
df = train_small.copy()

In [70]:
df = df[~df['MISSING_DATA']].drop(columns='MISSING_DATA')

In [71]:
# Over every single 
def polyline_to_trip_duration(polyline):
    return max(polyline.count("[") - 2, 0) * 15

# This code creates a new column, "LEN", in our dataframe. The value is
# the (polyline_length - 1) * 15, where polyline_length = count("[") - 1
df["LEN"] = df["POLYLINE"].apply(polyline_to_trip_duration)

In [72]:
from datetime import datetime
def parse_time(x):
    # We are using python's builtin datetime library
    # https://docs.python.org/3/library/datetime.html#datetime.date.fromtimestamp

    # Each x is essentially a 1 row, 1 column pandas Series
    dt = datetime.fromtimestamp(x["TIMESTAMP"])
    return dt.year, dt.month, dt.day, dt.hour, dt.weekday()

# Because we are assigning multiple values at a time, we need to "expand" our computed (year, month, day, hour, weekday) tuples on 
# the column axis, or axis 1
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.apply.html
df[["YR", "MON", "DAY", "HR", "WK"]] = df[["TIMESTAMP"]].apply(parse_time, axis=1, result_type="expand")

In [73]:
mapper = {'A': 1, 'B': 2, 'C': 3}

In [74]:
df['CALL_TYPE'] = df['CALL_TYPE'].replace(mapper)

In [75]:
metadata = pd.read_csv('metaData_taxistandsID_name_GPSlocation.csv')

In [76]:
metadata['coords'] = '[' + metadata['Longitude'].astype(str) + ', ' + metadata['Latitude'].astype(str) + ']'

In [77]:
# df['START_COORD'] = df['POLYLINE'].apply(lambda x: eval(x)).str[0]

In [78]:
metadata['Longitude'][40] = metadata['Latitude'][40][12:]
metadata['Latitude'][40] = metadata['Latitude'][40][0:12]

In [79]:
#df['ORIGIN_STAND'] = df['ORIGIN_STAND'].fillna(-1)

In [80]:
df.loc[df['CALL_TYPE'] == 1, 'ORIGIN_STAND'] = 0

In [81]:
df.loc[df['CALL_TYPE'] == 3, 'ORIGIN_STAND'] = -1

In [82]:
#df.groupby('CALL_TYPE')['LEN'].mean().plot(kind='bar', ylabel='MEAN LEN')

In [83]:
#driver_route_list = list(df.sort_values(by=['TAXI_ID', 'TIMESTAMP']).reset_index(drop=True).groupby('TAXI_ID')['ORIGIN_STAND'])

In [84]:
#driver_routes = {t[0]: list(t[1]) for t in driver_route_list}#

In [85]:
def find_closest_city(coords):
    #pass in START_COORDS
    if isinstance(coords, float):
        return None
    lat_dif = metadata['Latitude'].astype(float) - coords[1]
    long_dif = metadata['Longitude'].astype(float) - coords[0]
    distance = (lat_dif**2 + long_dif**2)**(1/2)
    return distance.idxmin() + 1

In [86]:
#df['CITY_ID'] = df['START_COORD'].apply(lambda x: find_closest_city(x))

In [87]:
from matplotlib.pyplot import figure

In [88]:
# import matplotlib.pyplot as plt
# plt.rcParams["figure.figsize"] = (20,10)

# df.groupby('ORIGIN_STAND')['LEN'].mean().plot(kind='barh')

In [89]:
df['DATESTAMP'] = pd.to_datetime(df['TIMESTAMP'], unit='s')

In [90]:
df['DAY_OF_WEEK'] = df['DATESTAMP'].transform(lambda x: str.upper(x.day_name()))

In [91]:
for day in df['DAY_OF_WEEK'].unique():
    df[day] = (df['DAY_OF_WEEK'] == day).astype(int)

In [92]:
# building training dataset

In [93]:
train_df = df[['CALL_TYPE', 'ORIGIN_STAND', 'MON', 'DAY_OF_WEEK', 'HR']].copy()

In [94]:
for call_type in train_df['CALL_TYPE'].unique():
    train_df['CALL_TYPE' + ': ' + str(call_type)] = (train_df['CALL_TYPE'] == call_type).astype(int)
        
# excludes city 63 to prevent multicollinearity
for origin_id in list(range(-1, 63)):
    train_df['ORIGIN_STAND: ' + str(origin_id)] = (train_df['ORIGIN_STAND'] == origin_id).astype(int)

# exclude December to prevent multicollinearity
for mon in list(range(1,12)):
    train_df['MON: ' + str(mon)] = (train_df['MON'] == mon).astype(int)

# all 0 = sunday
for day in ['MONDAY', 'TUESDAY', 'WEDNESDAY', 'THURSDAY', 'FRIDAY', 'SATURDAY']:
    train_df['DAY_OF_WEEK: ' + day] = (train_df['DAY_OF_WEEK'] == day).astype(int)

In [95]:
train_df

,CALL_TYPE,ORIGIN_STAND,MON,DAY_OF_WEEK,HR,CALL_TYPE: 1,CALL_TYPE: 3,CALL_TYPE: 2,ORIGIN_STAND: -1,ORIGIN_STAND: 0,...,MON: 8,MON: 9,MON: 10,MON: 11,DAY_OF_WEEK: MONDAY,DAY_OF_WEEK: TUESDAY,DAY_OF_WEEK: WEDNESDAY,DAY_OF_WEEK: THURSDAY,DAY_OF_WEEK: FRIDAY,DAY_OF_WEEK: SATURDAY
1479628,1,0.0,5,FRIDAY,12,1,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
1426923,3,-1.0,5,TUESDAY,14,0,1,0,1,0,...,0,0,0,0,0,1,0,0,0,0
613252,3,-1.0,11,SATURDAY,3,0,1,0,1,0,...,0,0,0,1,0,0,0,0,0,1
1553683,3,-1.0,5,SATURDAY,8,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,1
1218007,3,-1.0,3,SUNDAY,2,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1361784,3,-1.0,4,WEDNESDAY,16,0,1,0,1,0,...,0,0,0,0,0,0,1,0,0,0
173706,2,42.0,8,WEDNESDAY,19,0,0,1,0,0,...,1,0,0,0,0,0,1,0,0,0
1239954,3,-1.0,3,FRIDAY,7,0,1,0,1,0,...,0,0,0,0,0,0,0,0,1,0
989284,1,0.0,2,SATURDAY,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1


In [96]:
train_df = train_df.iloc[:, 4:].sort_index(axis=1)

In [97]:
train_df = train_df.reset_index(drop=True)

In [98]:
# append col of 1s, intercept
train_df = pd.concat([train_df, pd.Series(np.repeat(1, train_df.shape[0]))], axis=1)

In [99]:
X_train = np.array(train_df)

In [100]:
y_train = df['LEN']

In [101]:
y_train.shape

(799994,)

In [102]:
regularizer = .01

In [103]:
w = np.linalg.inv(X_train.T @ X_train + regularizer * np.eye((X_train.T@X_train).shape[0])) @ X_train.T @ y_train

In [104]:
train_MSE = (1/len(y_train))*(sum((w @ X_train.T - y_train)**2))**(1/2)
train_MSE

0.7591508512968447

In [105]:
test = pd.read_csv('kaggle_data/test_public.csv')

In [106]:
test['CALL_TYPE'] = test['CALL_TYPE'].replace(mapper)

In [107]:
test.loc[test['CALL_TYPE'] == 1, 'ORIGIN_STAND'] = 0

In [108]:
test.loc[test['CALL_TYPE'] == 3, 'ORIGIN_STAND'] = -1

In [109]:
from datetime import datetime
def parse_time(x):
    # We are using python's builtin datetime library
    # https://docs.python.org/3/library/datetime.html#datetime.date.fromtimestamp

    # Each x is essentially a 1 row, 1 column pandas Series
    dt = datetime.fromtimestamp(x["TIMESTAMP"])
    return dt.year, dt.month, dt.day, dt.hour, dt.weekday()

# Because we are assigning multiple values at a time, we need to "expand" our computed (year, month, day, hour, weekday) tuples on 
# the column axis, or axis 1
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.apply.html
test[["YR", "MON", "DAY", "HR", "WK"]] = test[["TIMESTAMP"]].apply(parse_time, axis=1, result_type="expand")

In [110]:
test['DATESTAMP'] = pd.to_datetime(test['TIMESTAMP'], unit='s')

In [111]:
test['DAY_OF_WEEK'] = test['DATESTAMP'].transform(lambda x: str.upper(x.day_name()))

In [112]:
test_df = test[['CALL_TYPE', 'ORIGIN_STAND', 'MON', 'DAY_OF_WEEK', 'HR']].copy()

In [113]:
for call_type in test_df['CALL_TYPE'].unique():
    test_df['CALL_TYPE' + ': ' + str(call_type)] = (test_df['CALL_TYPE'] == call_type).astype(int)
        
# excludes city 63 to prevent multicollinearity
for origin_id in list(range(-1, 63)):
    test_df['ORIGIN_STAND: ' + str(origin_id)] = (test_df['ORIGIN_STAND'] == origin_id).astype(int)

# exclude December to prevent multicollinearity
for mon in list(range(1,12)):
    test_df['MON: ' + str(mon)] = (test_df['MON'] == mon).astype(int)

# all 0 = sunday
for day in ['MONDAY', 'TUESDAY', 'WEDNESDAY', 'THURSDAY', 'FRIDAY', 'SATURDAY']:
    test_df['DAY_OF_WEEK: ' + day] = (test_df['DAY_OF_WEEK'] == day).astype(int)  

In [114]:
X_test = test_df.iloc[:, 4:].sort_index(axis=1)

In [115]:
# append col of 1s, intercept
X_test = pd.concat([X_test, pd.Series(np.repeat(1, X_test.shape[0]))], axis=1)

In [116]:
X_test = np.array(X_test)

In [117]:
submission = pd.DataFrame(np.dot(X_test, w))

In [118]:
submission.columns = ['TRAVEL_TIME']

In [119]:
submission.index = test['TRIP_ID']

In [120]:
submission

,TRAVEL_TIME
TRIP_ID,
T1,767.506917
T2,651.234160
T3,767.506917
T4,622.325896
T5,690.877603
...,...
T323,680.010492
T324,567.429716
T325,720.052345


In [121]:
submission.to_csv('submission.csv')
